TODOs
- Baseline timings for each model-opt combination (time-per-token not using guidance)
- Same timings but with guidance
- Look at a set of combinations of model and "optimisation stategy"

In [1]:
import torch
torch.cuda.empty_cache()
from mlisplacement.struc_extract import DomainVariable, MathOperator, Comparator, Units, ValueNode, VariableNode, BinaryOpNode, ComparisonNode, ConditionalNode, AnyNode, Node
from mlisplacement.struc_extract import airports
from mlisplacement.struc_extract import reasoning
from mlisplacement.struc_extract import QwenChatTemplate
from mlisplacement.utils import load_model, read_md, timing_decorator

import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from enum import Enum
# from guidance import models, system, user, assistant, json as gen_json, gen
import torch
import json
import torch
from enum import Enum
import guidance
import os

In [2]:
datadir = os.path.join("..", "..", "data")
filepath = os.path.join(datadir, "struc_extract/output/2025-airport-charges-terms-and-conditions/tinychargesmarkdown.md")
# MODEL_ID = "Qwen/Qwen3-30B-A3B"
# MODEL_ID = "Qwen/Qwen3-30B-A3B-Instruct-2507"
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct" #This already has memory error
# MODEL_ID = "distilgpt2"

In [3]:
hf_model, hf_tokenizer = load_model(MODEL_ID, low_cpu_mem_usage=True)
tok = hf_tokenizer
chat_template = QwenChatTemplate
model = guidance.models.Transformers(hf_model, tok, chat_template=chat_template)

gpustat is not installed, run `pip install gpustat` to collect GPU stats.


In [ ]:
Var = create_dynamic_variable_enum("transfer_passenger_charge")
print(Var.TRANSFER_PASSENGER_COUNT.value)

In [ ]:
allowed_variables = [el.value for el in list(Var)]
allowed_variables

In [ ]:
# class ComputationGraphBuilder:
#     """
#     Orchestrates the creation of a computation graph by preparing dynamic
#     constraints and prompting the LLM.
#     """
    
#     def __init__(self, model):
#         """
#         Initializes the builder with a guidance model.
#         """
#         self.model = model
#         # Set the default LLM for all guidance programs
#         # guidance.llm = self.model
#     @timing_decorator
#     def build(self, document_content: str, query: str, charge_category: str) -> dict:
#         """
#         Generates a computation graph for a given query and document.

#         Args:
#             document_content: The text containing the rules.
#             query: A natural language question about what to calculate.
#             charge_category: The specific charge context used to filter variables.

#         Returns:
#             A dictionary representing the computation graph or an error.
#         """
#         print(f"--- Building graph for charge category: '{charge_category}' ---")
        
#         # 1. Dynamically create the filtered Enum for this specific task
#         try:
#             Var = create_dynamic_variable_enum(charge_category)
#         except ValueError as e:
#             print(f"Error: {e}")
#             return {"error": str(e)}

#         # 3. Create a formatted prompt string of allowed variables for the LLM
#         allowed_variables = [el.value for el in list(Var)]
#         allowed_variables_prompt = "\n".join(
#             [f"- **{v.name}**: {v.description}" for name, v in airports.ALL_VARIABLES.items() if name in allowed_variables]
#         )

#         try:
#             # 4. Execute the guidance program with all dynamic components
#             result_lm = self.model + create_graph_with_cot(
#                 allowed_variables_prompt=allowed_variables_prompt,
#                 document=document_content,
#                 query=query,
#                 output_schema=Node
#             )
            
#             return result_lm
            
#         except Exception as e:
#             print(f"\nAn error occurred while building the graph for '{query}': {e}")
#             return {"error": str(e)}

In [ ]:
import sympy
from sympy import Symbol, Piecewise, sympify, Add, Mul, Pow

def compose_expression(node: dict):
    """
    Recursively parses a JSON graph into a SymPy expression,
    preventing automatic simplification.

    Args:
        node: A dictionary representing a node in the computation graph.

    Returns:
        A non-evaluated sympy expression representing the computation.
    """

    node_type = node.get('type')

    if node_type == "VALUE":
        return sympify(node['value'])

    elif node_type == "VARIABLE":
        return Symbol(node['name'])

    elif node_type == "BINARY_OPERATION":
        left = compose_expression(node['left'])
        right = compose_expression(node['right'])
        operator = node['operator']
        
        # Use class constructors with evaluate=False to prevent simplification
        if operator == "ADD":
            return Add(left, right, evaluate=False)
        elif operator == "MULTIPLY":
            return Mul(left, right, evaluate=False)
        elif operator == "DIVIDE":
            # Division (a/b) is represented as a * (b**-1)
            power = Pow(right, -1, evaluate=False)
            return Mul(left, power, evaluate=False)
        else:
            raise ValueError(f"Unsupported binary operator: {operator}")

    elif node_type == "COMPARISON":
        left = compose_expression(node['left'])
        right = compose_expression(node['right'])
        operator = node['operator']

        if operator == "GREATER_THAN":
            return left > right
        if operator == "LESS_THAN":
            return left < right
        else:
            raise ValueError(f"Unsupported comparison operator: {operator}")

    elif node_type == "CONDITIONAL":
        condition = compose_expression(node['condition'])
        if_true_expr = compose_expression(node['if_true'])
        if_false_expr = compose_expression(node['if_false'])
        
        return Piecewise((if_true_expr, condition), (if_false_expr, True))

    else:
        raise ValueError(f"Unknown node type: {node_type}")

In [ ]:
import json
import traceback

#TODO: Rename to generate or similar?
# def create_computation_graph(model, query, charge_category, markdown_content):
#     graph_builder = ComputationGraphBuilder(model=model)

#     start_time = time.perf_counter()
#     llm_structured_response = graph_builder.build(
#         document_content=markdown_content,
#         query=query,
#         charge_category=charge_category
#     )
#     end_time = time.perf_counter()
#     build_time = end_time - start_time

#     return llm_structured_response, build_time

def generate(model, query, charge_category, markdown_content):
    graph_builder = ComputationGraphBuilder(model=model)

    start_time = time.perf_counter()
    llm_structured_response = graph_builder.build(
        document_content=markdown_content,
        query=query,
        charge_category=charge_category
    )
    end_time = time.perf_counter()
    build_time = end_time - start_time

    return llm_structured_response, build_time

    

In [ ]:
datadir = os.path.join("..", "..", "data")
filepath = os.path.join(datadir, "struc_extract/output/2025-airport-charges-terms-and-conditions/tinychargesmarkdown.md")
# MODEL_ID = "Qwen/Qwen3-30B-A3B"
# MODEL_ID = "Qwen/Qwen3-30B-A3B-Instruct-2507"
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct" #This already has memory error
# MODEL_ID = "distilgpt2"

query = "Calculate the total parking charge for a Narrow Satellite stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

res_model, build_time = generate(model, query, charge_category, markdown_content=input_md)


In [ ]:
def export_data():
    pass

In [ ]:

# Print the resulting expression
loaded_json = json.loads(res_model[0]['result_graph'])
symbolic_expr = compose_expression(loaded_json)
print(f"Sympy Expression: {symbolic_expr}")
export_data()

In [ ]:
result

In [ ]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

In [ ]:
result[1]

In [ ]:



# Print the resulting expression
loaded_json = json.loads(result[0]['result_graph'])
symbolic_expr = compose_expression(loaded_json)
print(f"Sympy Expression: {symbolic_expr}")


# Changes: 
- Tried on a new problem

# Observations: 
- The reasoning traces are decent, except that I may need to add a ceil function to account for cases when one may need to round up.
- The graph produced is not perfect. It makes a potential mistake in computing surcharge, however, at this point, it is not clear whether this is a mistake as the document is quite confusing by itself. It does not clarify whether the surcharge is applicable for Long Term Remote, and whether that is on a per day basis.
- The left hand side branch does not use the variable parking hours to compute the surcharge even in cases where the hours are > 24. One potential reason for this could be that I mentioned there is no overnight parking, which may have led it to assume that the hours would be less than 24.
- The overnight parking charges also have ambiguity in their application. It is not clear, whether one has to discount the free period from the computed total fees or whether the surcharges are still applicable during that period, or it could be that aircrafts which strictly use the airport services starting and ending strictly during this period are exempt from any fees? 

# Discussion:
- More testing is required to understand where the model may be making mistakes.
- Larger models may perform better, as we currently only use the humble 32B A3B parameter MoE model.

# Next Steps:
- Continue testing

In [ ]:
query = "Calculate the total parking charge for a Long Term Remote stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


In [ ]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

In [ ]:
def export_results(model: guidance.models.Transformers, data_dir):
    #TODO: Export to json


# Changes: 
- Tried on a new problem

# Observations: 

# Discussion:

# Next Steps:


In [ ]:
query = "Calculate the total parking charge for a Wide Satellite stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


In [ ]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

In [ ]:
query = "Calculate the total parking charge for a Light Aircraft Parking (LAP) stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


In [ ]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

In [ ]:
result[0]["result_graph"]

In [ ]:
query = "Calculate the total parking charge for a Light Aircraft Parking (LAP) stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)


In [ ]:
print(json.dumps(json.loads(result[0]["thought4"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought3"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought2"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["thought1"]), indent=4))

In [ ]:
print(json.dumps(json.loads(result[0]["result_graph"]), indent=4))

In [ ]:
query = "Calculate the total parking charge for a Light Aircraft Parking (LAP) stand type at East Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
charge_category = "east_aerodrome_parking_charge"

result = create_computation_graph(model, query, charge_category)